# CoDeepNEAT demo
## CS081 project checkpoint demo
### Harsha Uppli, Alan Zhao, Gabriel Meyer-Lee

The following notebook demonstrates using CoDeepNEAT to solve MNIST

In [1]:
from math import pi, floor
from random import random
from codeepneat import config, population, chromosome, genome, visualize
import pickle
import numpy as np
import keras

Using TensorFlow backend.


AttributeError: module 'codeepneat.genome' has no attribute 'ConnectionGene'

### Calculating Coverage

The following **Grid** class is used to create a fitness function for calculating how well a robot's motion covers a given environment. To construct a grid, you must pass in the desired grid width and the environment's width.  Make sure you understand this class and its methods.

In [2]:
class Grid(object):
    """This class creates a grid of locations on top of a simulated world
    to monitor how much of the world has been visited. Each grid location
    is initally set to 0 to indicate that it is unvisited, and is updated
    to 1, once it has been visited."""
    def __init__(self, grid_width, world_width):
        self.grid_width = grid_width
        self.world_width = world_width
        self.grid = []
        for i in range(self.grid_width):
            self.grid.append([0] * self.grid_width)

    def show(self):
        """Print a representation of the grid."""
        for i in range(self.grid_width):
            for j in range(self.grid_width):
                print("%2d" % self.grid[i][j], end=" ")
            print()
        print()
        
    def update(self, x, y):
        """In the simulator, the origin is at the bottom left corner.
        Adjust the row to match this configuration. Set the appropriate
        grid location to 1."""
        size = self.world_width/self.grid_width
        col = floor(x/size)
        # adjust the row so that it matches the simulator
        row = self.grid_width - 1 - floor(y/size)
        self.grid[row][col] = 1
        
    def analyze_visits(self):
        """Calculate the percentage of visited cells in the grid."""
        cells_visited = 0
        for i in range(self.grid_width):
            for j in range(self.grid_width):
                if self.grid[i][j] > 0:
                    cells_visited += 1
        percent_visited = cells_visited/self.grid_width**2
        return percent_visited


### Testing Coverage

Let's create a world and robot.  We'll allow the robot to move around randomly.  As it moves, we can record the (x, y) locations that it visits. Test the grid several times.  Try different grid sizes, such as 10 or 20.

In [3]:
def make_world(physics):
    physics.addBox(0, 0, 4, 4, fill="white", wallcolor="black")

In [4]:
def make_robot():
    robot = Pioneer("Pioneer", 2, 2, 0) #paremeters are x, y, heading (in radians)
    robot.addDevice(Pioneer16Sonars())
    return robot

In [5]:
def random_act():
    return random()*2-1, random()*2-1

In [6]:
def test_grid():
    robot = make_robot()
    vsim = VSimulator(robot, make_world)
    robot.useTrail = True
    robot.display['trail'] = 1
    grid = Grid(10, 4)
    action = random_act()
    for i in range(100):
        vsim.step()
        if i%20 == 0:
            action = random_act()
        robot.move(*action)
        x, y, a = robot.getPose()
        grid.update(x, y)
    grid.show()
    percent =  grid.analyze_visits()
    print("Percent visited", percent) 

test_grid()

VBox(children=(VBox(children=(HBox(children=(Checkbox(value=True, description='Update GUI'), Checkbox(value=False, description='Trace Path'), Text(value='0.0 seconds', description='Time:'))), HBox(children=(VBox(children=(HBox(children=(FloatSlider(value=0.0, layout=Layout(height='248px', padding='0px 0px 0px 0px', width='10px'), orientation='vertical', readout=False), HTML(value='<svg baseProfile="full" height="240" version="1.1" width="240" xmlns="http://www.w3.org/2000/svg" xmlns:ev="http://www.w3.org/2001/xml-events" xmlns:xlink="http://www.w3.org/1999/xlink"><defs /><rect fill="#ffffff" height="240.0" stroke="#000000" stroke-width="1" width="240.0" x="0.0" y="0.0" /><line fill="#800080" stroke="#add8e6" stroke-width="1" x1="109.5" x2="0.0" y1="114.0" y2="114.0" /><line fill="#800080" stroke="#add8e6" stroke-width="1" x1="111.0" x2="0.0" y1="109.80000000000001" y2="58.039848000000006" /><line fill="#800080" stroke="#add8e6" stroke-width="1" x1="113.4" x2="22.777240194617416" y1="107.99999999999999" y2="0.0" /><line fill="#800080" stroke="#add8e6" stroke-width="1" x1="117.0" x2="88.4634107462688" y1="106.5" y2="0.0" /><line fill="#800080" stroke="#add8e6" stroke-width="1" x1="122.99999999999999" x2="151.53658925373122" y1="106.5" y2="0.0" /><line fill="#800080" stroke="#add8e6" stroke-width="1" x1="126.6" x2="217.22275980538257" y1="107.99999999999999" y2="0.0" /><line fill="#800080" stroke="#add8e6" stroke-width="1" x1="129.0" x2="240.0" y1="109.80000000000001" y2="58.039848000000006" /><line fill="#800080" stroke="#add8e6" stroke-width="1" x1="130.5" x2="240.0" y1="114.0" y2="114.0" /><line fill="#800080" stroke="#add8e6" stroke-width="1" x1="130.5" x2="240.0" y1="126.0" y2="126.0" /><line fill="#800080" stroke="#add8e6" stroke-width="1" x1="129.0" x2="240.0" y1="130.2" y2="181.960152" /><line fill="#800080" stroke="#add8e6" stroke-width="1" x1="126.6" x2="217.22275980538257" y1="132.0" y2="240.0" /><line fill="#800080" stroke="#add8e6" stroke-width="1" x1="122.99999999999999" x2="151.53658925373122" y1="133.5" y2="240.0" /><line fill="#800080" stroke="#add8e6" stroke-width="1" x1="117.0" x2="88.4634107462688" y1="133.5" y2="240.0" /><line fill="#800080" stroke="#add8e6" stroke-width="1" x1="113.39999999999999" x2="22.777240194617413" y1="132.0" y2="240.0" /><line fill="#800080" stroke="#add8e6" stroke-width="1" x1="111.0" x2="0.0" y1="130.2" y2="181.960152" /><line fill="#800080" stroke="#add8e6" stroke-width="1" x1="109.5" x2="0.0" y1="126.0" y2="126.0" /><polygon fill="red" points="115.19999999999999,106.5 109.5,111.0 109.5,129.0 115.19999999999999,133.5 124.80000000000001,133.5 130.5,129.0 130.5,111.0 124.80000000000001,106.5" stroke="red" stroke-width="1" /><polygon points="-3.0,-3.0 0,0 3.0,-3.0 0,3.0" stroke="black" stroke-width="1" transform="translate(120.0,120.0) rotate(180.0)" /></svg>'))), FloatSlider(value=0.0, layout=Layout(height='30px', padding='0px 0px 0px 10px', width='260px'), readout=False))), VBox(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00x\x00\x00\x00<\x08\x02\x00\x00\x00"\xcf\xe9\xff\x00\x00\x00+IDATx\x9c\xed\xc1\x01\x01\x00\x00\x00\x82 \xff\xafnH@\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x8f\x06T\x9c\x00\x01f.\xa6\x1d\x00\x00\x00\x00IEND\xaeB`\x82', width='240'), FloatSlider(value=0.0, layout=Layout(height='20px', padding='0px 0px 0px 0px', width='248px'), readout=False))))))), HBox(children=(Button(icon='step-forward', style=ButtonStyle()), Play(value=0, max=1000000), Button(description='Clear Output', style=ButtonStyle()))), Output()))

 0  0  0  0  0  0  0  0  0  0 
 0  0  0  0  0  0  0  0  0  0 
 0  0  0  0  0  0  0  0  0  0 
 0  0  0  0  0  0  0  0  1  0 
 0  0  0  0  0  1  0  0  1  0 
 0  0  0  0  1  1  1  0  1  1 
 0  0  0  0  0  0  1  1  0  1 
 0  0  0  0  0  0  0  1  1  1 
 0  0  0  0  0  0  0  1  1  0 
 0  0  0  0  0  0  0  0  0  0 

Percent visited 0.16


### Configuring NEAT

Set key parameters of the NEAT experiments here.  

In [7]:
%%file configMNIST
#--- parameters for the robot experiment ---#
[phenotype]
input_nodes         = 4
output_nodes        = 2
max_weight          = 30
min_weight          = -30
feedforward         = 1
nn_activation       = tanh
hidden_nodes        = 0
weight_stdev        = 0.9

[genetic]
pop_size              = 50
max_fitness_threshold = 1

# Human reasoning
prob_addconn          = 0.1
prob_addnode          = 0.05
prob_mutatebias       = 0.2
bias_mutation_power   = 0.5
prob_mutate_weight    = 0.95
weight_mutation_power = 1.5
prob_togglelink       = 0.01
elitism               = 1

[genotype compatibility]
compatibility_threshold = 3.0
compatibility_change    = 0.0
excess_coeficient       = 1.0
disjoint_coeficient     = 1.0
weight_coeficient       = 0.4

[species]
species_size        = 10
survival_threshold  = 0.2
old_threshold       = 30
youth_threshold     = 10
old_penalty         = 0.2
youth_boost         = 1.2
max_stagnation      = 15

Overwriting configCoverage


### Sensor inputs for achieving coverage

What are the best sensors for allowing NEAT to find good coverage solutions? Here are some options to consider:

1. all or some subset of the scaled sonars
2. stall
3. scaled timer (either counting up or counting down)
4. bias (some constant value)
5. current coverage

For example, the following function uses just three sensors: the scaled minimum of the front sonar sensors, the stall sensor, and a count down timer.  The count down timer gives the robot a sense of how long a trial will last. A count down timer starts at 1.0 and decreases linearly to 0.0. A count up timer works in the reverse direction. It starts at 0.0 and increases linearly to 1.0. 

Remember that all inputs and targes to a neural network should be scaled to match the range of the activation function.  In our case, this will be **tanh** which has a range of **[-1,1]**.

In [8]:
def get_sensors(robot, steps, cover, i):
    sonars = robot["sonar"].getData()
    scaled = [min(v/5.0, 1.0) for v in sonars]
    timer_down = (steps-i)/steps
    inputs = []
    inputs.append(min(sonars[3:5]))
    #inputs.append(min(sonars[7:9]))
    #inputs.append(min(sonars[11:13]))
    #inputs.append(min(sonars[15], sonars[0]))
    inputs.append(robot.stall) 
    inputs.append(timer_down)
    inputs.append(cover)
    return inputs

In [9]:
robot = make_robot()
get_sensors(robot, 1000, .5, 100)

[0, 0, 0.9, 0.5]

### Fitness

We will tie a network's fitness directly to the percent of grid locations a robot visits. Notice that on each time step, the **eval_individual** function is running a non-visual simulatator and uses the **get_sensors** function to determine the inputs to the networks.

In [10]:
def eval_individual(brain, robot, sim, show_trail=False, steps=1000):
    robot.setPose(2, 2, 0)
    if show_trail:
        robot.useTrail = True
        robot.trail = []   
        robot.display['trail'] = 1
    grid = Grid(15, 4) 
    for i in range(steps):
        brain.flush()
        cover = grid.analyze_visits()
        inputs = get_sensors(robot, steps, cover, i)
        output = brain.sactivate(inputs)
        robot.move(*output)
        x, y, a = robot.getPose()
        grid.update(x, y)
        sim.step()
    return grid.analyze_visits()

In [11]:
def eval_population(population):
    print("Evaluating chromo", end=" ")
    for i in range(len(population)):
        print(i, end=" ")
        chromo = population[i]
        brain = nn.create_ffphenotype(chromo)
        chromo.fitness = eval_individual(brain, robot, sim)
    print()

### Evolution

Note that evolving brains for robots will be a **much** slower process than evolving solutions to XOR.  Here we need to create the network and allow it to control the robot for an extended period of time in the simulator in order to determine it's fitness. In order for you to see quick results, I have initially set the number of generations to 5, however, you should increase this for your experiments. 

Remember to look at the files **avg_fitness.svg** and **speciation.svg** after evolution is complete. You'll need to use a linux command such as **eog** in a terminal window to view them.

In [12]:
def evolve(n):
    config.load('configCoverage')
    chromosome.node_gene_type = genome.NodeGene
    # Tell NEAT that we want to use the above function to evaluate fitness
    population.Population.evaluate = eval_population
    # Create a population (the size is defined in the configuration file)
    pop = population.Population()
    # Run NEAT's genetic algorithm for at most 30 epochs
    # It will stop if fitness surpasses the max_fitness_threshold in config file
    pop.epoch(n, report=True, save_best=True, name="Coverage")
    # Plots the evolution of the best/average fitness
    visualize.plot_stats(pop.stats, name="Coverage_")
    # Visualizes speciation
    visualize.plot_species(pop.species_log, name="Coverage_")

In [13]:
robot = make_robot()
sim = Simulator(robot, make_world)
evolve(10)


 ****** Running generation 0 ****** 

Evaluating chromo 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 
Population's average fitness: 0.10213 stdev: 0.07805
Best fitness: 0.3155555555 - size: (0, 8) - species 1 - id 3
Species length: 1 totalizing 50 individuals
Species ID       : [1]
Each species size: [50]
Amount to spawn  : [50]
Species age      : [0]
Species no improv: [0]

 ****** Running generation 1 ****** 

Evaluating chromo 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 
Population's average fitness: 0.18436 stdev: 0.13464
Best fitness: 0.5866666666 - size: (0, 8) - species 1 - id 72
Species length: 2 totalizing 50 individuals
Species ID       : [1, 2]
Each species size: [49, 1]
Amount to spawn  : [15, 35]
Species age      : [1, 0]
Species no improv: [0, 0]

 ****** Running generation 2 ****** 

In [14]:
def eval_best(chromo_file):
    config.load('configCoverage')
    chromosome.node_gene_type = genome.NodeGene
    fp = open(chromo_file, "rb")
    chromo = pickle.load(fp)
    print(chromo)
    fp.close()
    visualize.draw_net(chromo, "_" + chromo_file)    
    brain = nn.create_ffphenotype(chromo)
    fitness = eval_individual(brain, robot, sim, show_trail=True)
    canvas = Canvas((400,400))
    sim.physics.draw(canvas)
    canvas.save("trail_" + chromo_file + ".svg")
    print("fitness", fitness)

### Analysis

Try evaluating the best chromosomes found at each generation of one of your evolutionary runs.  Describe how the behavior improved from one generation to the next.  Remember to look at the phenotype images that are also created when you evaluate the best chromosomes. These images show the topologies of the evolved networks. Describe whether the network topology changed from one generation to the next. 

### Answer

Fitness generally increased over the generations. However, some generations had a decrease in fitness which then allowed subsequent generations to increase their fitness. Network topologies sometimes changed dramatically from one generation to the next, although later generations tended to have less dramatic changes in topology.

In [15]:
eval_best("Coverage_best_chromo_8")

Nodes:
	Node  1  INPUT, bias  0, response 4.924273
	Node  2  INPUT, bias  0, response 4.924273
	Node  3  INPUT, bias  0, response 4.924273
	Node  4  INPUT, bias  0, response 4.924273
	Node  5 OUTPUT, bias  0, response 4.924273
	Node  6 OUTPUT, bias  0, response 5.71823472
Connections:
	In  1, Out  5, Weight +1.17630, Enabled, Innov 1
	In  2, Out  5, Weight -4.56807, Enabled, Innov 2
	In  3, Out  5, Weight -2.84179, Enabled, Innov 3
	In  4, Out  5, Weight +3.30274, Enabled, Innov 4
	In  1, Out  6, Weight +2.70763, Enabled, Innov 5
	In  2, Out  6, Weight +0.43135, Enabled, Innov 6
	In  3, Out  6, Weight -7.90194, Enabled, Innov 7
	In  4, Out  6, Weight -0.48354, Enabled, Innov 8
Node order: []
fitness 0.84


### Experiments

When using NEAT, many parameter settings must be specified and it isn't always clear how best to make these choices.  Each of the choices made in the code above can affect the types of behavior that will be evolved.  Below are just some of the parameter settings that you could explore. 

1. What inputs should be provided? In the original code only three were given: the normalized minimum of front sonar sensors, the stall sensor, and a count down timer. Try other combinations of inputs.

2. How should fitness be measured? In the original code we used a grid size of 15 and the percent visited as the fitness.  How would things change if we made the grid size bigger or smaller?  Is there a better method of evaluating coverage rather than percent visited?

3. How should the NEAT parameters be set? Should the probability of adding nodes and connections be changed?  Should more or less speciation be encouraged?  Should we use bigger populations and run for more generations? 

Run a series of 10 experiments using the original set up that I provided.  Each evolution should be for at least 10 generations. Report on the best coverage achieved in each run, as well as the complexity of the best networks in terms of the number of nodes and connections in the network topology. Note that the evolution process is not guaranteed to always lead to ever increasing fitness levels.  Sometimes an earlier generation can find a higher performing solution than a later generation. 

Modify the parameters in one of the ways suggested above. Run a series of 10 more experiments. Compare these results to original results.  Try to find a set of parameters that outperforms the ones I provided. 

### Results

#### Original Set Up

Experiment | Best Coverage | Best Complexity
--- | --- | ---
1 | .809 | 5n + 6c
2 | .840 | 5n + 6c
3 | .867 | 5n + 6c
4 | .898 | 5n + 6c
5 | .871 | 5n + 6c
6 | .796 | 7n + 10c
7 | .853 | 6n + 8c
8 | .862 | 5n + 6c
9 | .853 | 6n + 8c
10 | .836 | 5n + 6c

![Performance of original network](trailog.svg)

As shown above, this set up has a tendency to initially drive in circles gradually moving to the side to cover some of the center before doing laps around the parameter


We decided to make two small modifications to the hyperparameters: we added the current coverage of the board to the robot's existing sensors and increased the population to 50.

#### With Coverage sensor (and Population = 50)

Experiment | Best Coverage | Best Complexity
--- | --- | ---
1 | .813 | 7n + 10c
2 | .813 | 6n + 8c
3 | .871 | 8n + 10c
4 | .809 | 7n + 9c
5 | .884| 6n + 8c
6 | .840 | 6n + 8c
7 | .876 | 6n + 8c
8 | .782 | 6n + 8c
9 | .813 | 6n + 8c
10 | .800 | 8n + 11c
11 | .840 | 6n + 8c 

This new set up produced some instances of the previous behavior, as shown below.
![Performance of modified network](trailmod3.svg)

However, the best performing network we trained, trial 5, produced some novel, sharp turning behavior leading to a greater, non overlapping coverage.
![Performance of Trial 5 network](trailmod4.svg)

This behavior was not unique to our best trial, and was also identified in some lower performing trials such as Trial 1 and Trial 9.
![Performance of Trial 9 network](trailmod5.svg)
We also ran a few trials with the coverage sensor but over 30 generations instead of 10. This did produce a network with no hidden nodes that had a fitness of .907 and a network with 5 hidden nodes and a fitness of .876, but the trails and ultimate fitness functions of these networks demonstrated that the substantially longer training period bought only an incremental increase, if any, in performance.


### Use git to commit, add, and push

Be sure to save this notebook to the repository.